In [ ]:
#!/usr/bin/env python3
import pathlib
import base64
import requests
from openai import OpenAI

# Configuração
API_KEY = "a2c4457ed6a14299a425dd670e5a8ad0"
client = OpenAI(api_key=API_KEY, base_url="https://api.aimlapi.com/v1")

def save_from_url(url: str, out_path: pathlib.Path):
    r = requests.get(url, stream=True)
    r.raise_for_status()
    out_path.parent.mkdir(parents=True, exist_ok=True)
    with open(out_path, "wb") as f:
        for chunk in r.iter_content(8192):
            if chunk:
                f.write(chunk)

# Teste simples: 1 BG + 1 escudo
bg_file = "bgs/bg1.png"  # escolha um BG
logo_file = "stored_images/flamengo.svg"  # escolha um escudo
out_file = pathlib.Path("teste_resultado.png")

print(f"🔄 Testando: {bg_file} + {logo_file}")

# Abre as duas imagens
images = [open(bg_file, "rb"), open(logo_file, "rb")]

print(f"🔍 Tipo das imagens: {type(images)}")
print(f"🔍 Tamanho da lista: {len(images)}")
print(f"🔍 Tipo item 1: {type(images[0])}")
print(f"🔍 Tipo item 2: {type(images[1])}")

try:
    # Chamada EXATA igual ao notebook que funciona
    result = client.images.edit(
        model="openai/gpt-image-1",
        image=images,
        prompt="Quero esse fundo com a mesma forma e estilo porem com as cores desse escudo do escudo.png, coloque o logo com opacidade 50% para parecer integrado ao fundo",
        size="1024x1536",
        output_format="png",
        quality="medium",
        background="auto",
    )

    choice = result.data[0]
    print("✅ API respondeu com sucesso!")

    if getattr(choice, "url", None):
        save_from_url(choice.url, out_file)
        print(f"✅ Imagem salva em: {out_file}")
    elif getattr(choice, "b64_json", None):
        img_bytes = base64.b64decode(choice.b64_json)
        out_file.parent.mkdir(parents=True, exist_ok=True)
        with open(out_file, "wb") as f:
            f.write(img_bytes)
        print(f"✅ Imagem salva em: {out_file} (via base64)")
    else:
        print("⚠️ Resposta inesperada:", result)

except Exception as e:
    print(f"❌ Erro: {type(e).__name__}: {str(e)}")
    import traceback
    traceback.print_exc()

finally:
    for f in images:
        f.close()

print("🏁 Teste concluído")